## Import thư viện

In [ ]:
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

## Show kết quả quá trình training

In [ ]:
def plot_hist(path):
  history = pd.read_csv(path)

  acc = history['accuracy']
  val_acc = history['val_accuracy']

  loss = history['loss']
  val_loss = history['val_loss']
  plt.style.use('fivethirtyeight')
  plt.figure(figsize=(20, 10))

  plt.subplot(1, 2, 1)
  plt.plot(acc, label='Training Accuracy')
  plt.plot(val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.ylabel('Accuracy')
  plt.ylim([min(plt.ylim()), 1])
  plt.title('Training and Validation Accuracy')
  plt.xlabel('epoch')

  plt.subplot(1, 2, 2)
  plt.plot(loss, label='Training Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.ylabel('Categorical Crossentropy')
  plt.ylim([min(plt.ylim()), max(plt.ylim())])
  plt.title('Training and Validation Loss')

  plt.xlabel('epoch')
  plt.savefig('evaluation.jpg')
  plt.show()

In [ ]:
plot_hist("../input/efficientnetb0/FGVC8-efn-b0.log")

## Load model

In [ ]:
HEIGHT = 480
WIDTH = 480
CHANNELS = 3
CLASSES = 6
top_dropout_rate = 0.2
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
weights_path = "../input/efficientnetb0/FGVC8-efn-b0.h5"

In [ ]:
def get_model():
    base_model = efn.EfficientNetB0(include_top=False, weights=None, input_shape=(HEIGHT, WIDTH, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(top_dropout_rate)(x)
    outputs = Dense(CLASSES, activation='sigmoid')(x)
    
    return Model(base_model.input, outputs)

In [ ]:
from tensorflow import keras
model = keras.models.load_model(weights_path)

## Load data test

In [ ]:
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission

## Preprocessing test data

In [ ]:
def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(test_img+'/'+file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
def data_full_augment(image):
    
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    flag = False

    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
        flag = True
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
        flag = True
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        flag = True


    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        flag = True
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
        flag = True
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
        flag = True
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90
        flag = True
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
        flag = True
    elif p_crop > .4:
        HEIGHT1 = image.shape[0]
        WIDTH1 = image.shape[1]
        crop_size_h = tf.random.uniform([], int(HEIGHT1*.8), HEIGHT1, dtype=tf.float32)
        crop_size_w = tf.random.uniform([], int(WIDTH1*.8), WIDTH1, dtype=tf.float32)
        image = tf.image.random_crop(image, size=[crop_size_h, crop_size_w, 3])
        flag = True
    
    return image

In [ ]:
def process(path):
    img = load_image(path)
    img = data_full_augment(img)
    img = tf.cast(img, tf.float32) / 255.0
    img = np.array(img)
    return cv2.resize(img , (480, 480)).reshape(-1, 480, 480, 3)

## Predict

In [ ]:
preds = []
for i in range(len(submission['image'])):
    test_images = submission['image'][i]
    pred = model.predict(process(test_images))[0]
    preds.append(pred)
    
preds = np.array(preds)

In [ ]:
perdict = (preds>0.2)
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    if len(temp) > 1 and "healthy" in temp:
        temp.remove("healthy")
    elif len(temp) == 0:
        temp.append('healthy')
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
submission['labels'] = np.array(answer)
submission

## Submission

In [ ]:
submission.to_csv('submission.csv', index=False)